In [1]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:46243")
client

<Client: 'tcp://127.0.0.1:46243' processes=2 threads=2, memory=7.00 GiB>

In [2]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

import os
import pickle

import SAM
import float_data as flt

import importlib
importlib.reload(SAM)
importlib.reload(flt)

from dask import delayed

In [3]:
model_folder = 'model'

In [4]:
#data = flt.retrieve_profiles(timeRange = slice('1995-01', '1995-02'))
#np.save('data/mask', data['n'])
#['r1i1p1f2', 'r3i1p1f2', 'r2i1p1f2', 'r8i1p1f2', 'r4i1p1f2', 'r5i1p1f3', 'r7i1p1f3', 'r9i1p1f2', 'r6i1p1f3', 'r17i1p1f2', 'r19i1p1f2', 'r18i1p1f2', 'r11i1p1f2', 'r16i1p1f2', 'r12i1p1f2', 'r10i1p1f2', 'r13i1p1f2', 'r14i1p1f2', 'r15i1p1f2']
mask = np.load('data/mask.npy', allow_pickle=True)

In [5]:
ids = ['r1i1p1f2', 'r3i1p1f2', 'r2i1p1f2', 'r8i1p1f2', 'r4i1p1f2', 'r5i1p1f3', 'r7i1p1f3', 'r9i1p1f2', 'r6i1p1f3', 'r10i1p1f2']
ids = ['r10i1p1f2']
BICs = {}
print(ids)

['r10i1p1f2']


In [6]:
for m_id in ids:
    path_id = '{}/{}'.format(model_folder, m_id)
    if not os.path.isdir(path_id):
        os.makedirs(path_id)
    print('Starting {}'.format(m_id))
    options = {'memberId' : m_id}
    data = flt.retrieve_profiles(timeRange = slice('1965-01', '1994-12'), mask=mask, options=options)
    
    data_sampled = flt.random_sample(data, 7000).compute()
    data_sampled = flt.normalise_data(data_sampled, 'N')
    pca = flt.train_pca(data_sampled, 3)
    
    data_trans = flt.pca_transform(data_sampled, pca).compute()
    
    bic = np.zeros(30)
    
    with open('{}/pca.obj'.format(path_id), 'wb') as file:
        pickle.dump(pca, file)
        file.close()
        
    print('Finished setup for {}'.format(m_id))
    
    for n_classes in range(7, 8):
        
        path_n = '{}/{}/{}'.format(model_folder, m_id, n_classes)
        if not os.path.isdir(path_n):
            os.makedirs(path_n)
        elif os.path.exists('{}/gmm.obj'.format(path_n)):
          pass
#            with open('{}/bic.obj'.format(path_n), 'rb') as file:
#                bic[n_classes] = pickle.load(file)
#                file.close()
#            continue
        
        gmm = flt.train_gmm(data_trans, n_classes)
        bic[n_classes] = gmm.bic(data_trans)
        
        with open('{}/gmm.obj'.format(path_n), 'wb') as file:
            pickle.dump(gmm, file)
            file.close()
            
        with open('{}/bic.obj'.format(path_n), 'wb') as file:
            pickle.dump(bic[n_classes], file)
            file.close()
        
        print('Finished {} with {} classes'.format(m_id, n_classes))
        
    BICs[m_id] = bic
    
with open('{}/BICs8-.obj'.format(model_folder), 'wb') as file:
    pickle.dump(BICs, file)
    file.close()

print('Done!')

Starting r10i1p1f2


/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/core/indexing.py:1228: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/core/indexing.py:1228: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


Finished setup for r10i1p1f2
Finished r10i1p1f2 with 7 classes
Done!
